To run this code, you need the followin python packages
* pyspark



In [59]:
import sys
import pyspark
from pyspark import SparkContext, SparkConf, SQLContext
from pyspark.sql.functions import col, size, explode

import os
os.environ["PYSPARK_PYTHON"]="python3"
os.environ["PYSPARK_DRIVER_PYTHON"]="python3"

class Tweet:
    hashtag = []
    location = []
    time = ""

In [60]:
# create Spark context with Spark configuration
conf = SparkConf().setAppName("Twitter data processing. Text file to RDD")
sc = SparkContext.getOrCreate(conf=conf)
    
sqlContext = SQLContext(sc)
df = sqlContext.read.json("stream_output_oct24.json")

df2 = df.withColumn("hashtag", explode("entities.hashtags"))
df2.createOrReplaceTempView("tweets")

In [156]:
#df.printSchema()
#df.show(1)

coor = sqlContext.sql("SELECT created_at, hashtag, id, coordinates  from tweets WHERE coordinates IS NOT Null")
coor.createOrReplaceTempView("coordinated")

place = sqlContext.sql("SELECT created_at, hashtag, id, coordinates, place  from tweets WHERE place IS NOT Null AND id NOT IN (SELECT id FROM coordinated)")
coor.createOrReplaceTempView("placed")

user_loc = sqlContext.sql("SELECT created_at, hashtag, id, coordinates, user.location as locat  from tweets WHERE user.location IS NOT Null AND id NOT IN (SELECT id FROM coordinated) AND id NOT IN (SELECT id FROM placed)")
coor.createOrReplaceTempView("user_located")

coorRDD = coor.rdd.map(lambda x: (x.hashtag.text, x))
placeRDD = place.rdd.map(lambda x: (x.hashtag.text, x))
user_locRDD = user_loc.rdd.map(lambda x: (x.hashtag.text, x))

#print(rdd.take(1))
groupRDD = rdd.groupByKey().map(lambda x : (x[0], list(x[1])))
#print(rdd.take(10))
pairs = groupRDD.filter(lambda x: len(x[1]) > 1)



In [107]:
pair2 = groupRDD.map(lambda x: (x[0], len(x[1]) )).sortBy(lambda s: s[1], 0)
print(pair2.take(5))

[('StandWithHongKong', 1299), ('HongKong', 1211), ('StandUpForCatalonia', 382), ('Chinese', 339), ('StandWithHK', 335)]


In [189]:
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
import time
geolocator = Nominatim(user_agent="Tweet Tracking Porject")
def find_location(address):
    try:
        location = geolocator.geocode(address)
        if location != None:
            return [location.latitude, location.longitude]
        else:
            return None
    except GeocoderTimedOut:
        time.sleep(1)
        return find_location(address)
    

In [190]:
coordinates = coorRDD.map(lambda x: {
    "hashtag":x[0], 
    "long": x[1].coordinates.coordinates[0], 
    "lat": x[1].coordinates.coordinates[1], 
    "created": x[1].created_at})
print(coordinates.take(1))

coordinates = placeRDD.map(lambda x: {
    "hashtag":x[0], 
    "long": x[1].place.bounding_box['coordinates'][0][0][0], 
    "lat": x[1].place.bounding_box['coordinates'][0][0][1], 
    "created": x[1].created_at})
print(coordinates.take(1))

coordinates2 = user_locRDD.map(lambda x: {
    "hashtag":x[0], 
    "location": find_location( x[1].location), 
    "created": x[1].created_at})
coordinates = coordinates2.map(lambda x: {
    "hashtag":x['hashtag'], 
    "long": x['location'][0], 
    "lat": x['location'][1], 
    "created": x['created']})
print(coordinates.take(1))


[{'hashtag': 'business', 'long': 114.1882, 'lat': 22.3224, 'created': 'Thu Oct 24 08:34:59 +0000 2019'}]
[{'hashtag': 'FreeHongKong', 'long': 114.149429, 'lat': 22.287976, 'created': 'Thu Oct 24 09:00:18 +0000 2019'}]
[{'hashtag': 'humanrights', 'long': 22.2793278, 'lat': 114.1628131, 'created': 'Thu Oct 24 08:27:33 +0000 2019'}]


Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 214.0 failed 1 times, most recent failure: Lost task 0.0 in stage 214.0 (TID 706, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/lib/python3.6/urllib/request.py", line 1318, in do_open
    encode_chunked=req.has_header('Transfer-encoding'))
  File "/usr/lib/python3.6/http/client.py", line 1254, in request
    self._send_request(method, url, body, headers, encode_chunked)
  File "/usr/lib/python3.6/http/client.py", line 1300, in _send_request
    self.endheaders(body, encode_chunked=encode_chunked)
  File "/usr/lib/python3.6/http/client.py", line 1249, in endheaders
    self._send_output(message_body, encode_chunked=encode_chunked)
  File "/usr/lib/python3.6/http/client.py", line 1036, in _send_output
    self.send(msg)
  File "/usr/lib